### stacks, queues, double-ended queues

#### stack
A stack is a collection of objects that are inserted and removed according to the last-in, first-out (LIFO) principle.
栈是一个后进先出的对象
A user may insert objects into a stack at any time, but may only access or remove the most recently inserted object that remains(at the so-called “top” of the stack).
永远弹出最近插入的对象
![Screenshot from 2017-03-23 18-57-40.png](https://ooo.0o0.ooo/2017/03/23/58d3aa3556dd3.png)

stack的抽象数据类型:
* s.push() 在栈顶增加元素
* s.pop()  在栈顶弹出元素
* s.top()  返回栈顶元素的值,但不删除它
* s.is_empty() 如果栈空返回true
* len(s)       返回栈中元素的个数

##### 基于数组实现一个栈

In [8]:
#更标准的实现应当像基于java的array,维持一个指向栈顶的指针,在push和pop是检查array的容量,
#必要时调整数组大小
#python中的list更像一个动态数组,append和pop简化了这一操作
class ArrayStack:
    def __init__(self):
        self.data=[]
        self.pointer=-1
        self.count=0
        
    def push(self,item):
        self.data.append(item)
        self.pointer+=1
        self.count+=1
        
    def pop(self):
        if self.count==0:
            raise Empty("no item")
        val=self.data[self.pointer]
        self.data.pop()
        self.pointer-=1
        self.count-=1
        return val
    def top(self):
        if self.count==0:
            raise Empty("no item")
        return self.data[self.pointer]
    
    def is_empty(self):
        return self.count==0
    
    def __len__(self):
        return self.count
    
    def __str__(self):
        return str(self.data)
    
a=ArrayStack()
a.push(1)
print(a)
print(len(a))
a.pop()
print(a)
print(len(a))

[1]
1
[]
0


![Screenshot from 2017-03-23 19-34-05.png](https://ooo.0o0.ooo/2017/03/23/58d3b2c17a003.png)

简单的栈应用:10进制转8进制

In [15]:
b=ArrayStack()
s=ArrayStack()
b.push(3467)
top=b.pop()
while top//8 !=0:
    b.push(top//8)
    s.push(top%8)
    top=b.pop()
s.push(top%8)
while not s.is_empty():
    print(s.pop(),end="")

6613

逆波兰表达式

#### queues (队列)
as a queue is a collection of objects that are inserted and removed according to the first-in, first-out (FIFO) principle. That is, elements can be inserted at any time,but only the element that has been in the queue the longest can be next removed.队列是先进后出
![Screenshot from 2017-03-23 20-11-43.png](https://ooo.0o0.ooo/2017/03/23/58d3bb9b9a938.png)

队列的抽象数据类型:
* Q.enquenue 进列
* Q.dequenue 出列 删除并返回第一个元素
* Q.first    返回第一个元素
* Q.is_empty  判断是否为空
* len(Q)     返回队列中元素个数

##### 基于数组实现一个队列

In [26]:
class Quene:
    DEFAULT_CAPACITY=1
    
    def __init__(self):
        self.pointer=0
        self.count=0
        self.data=[None]*self.DEFAULT_CAPACITY
        
    def enquenue(self,item):
        if self.count==len(self.data):
            self.resize(2*len(self.data))
        #队列循环,当pointer指向队列中最后一个元素时,这时再出列,则队列为空
        #因此将指针置零 
        lastpointer=(self.pointer+self.count)%len(self.data)
        self.data[lastpointer]=item
        self.count+=1
        
    def dequenue(self):
        if self.count==0:
            raise Empty("no item")
        item=self.data[self.pointer]
        self.data[self.pointer]=None
        self.pointer+=1
        self.count-=1
        #队列循环,当pointer指向队列中最后一个元素时,这是再出列,则队列为空
        #因此将指针置零
        self.pointer=(self.pointer+1)%len(self.data)
        return item
    def resize(self,size):
        olddata=self.data
        self.data=[None for i in range(size)]
        for i in range(len(olddata)):
            self.data[i]=olddata[i]
    
    def first(self):
        if self.count==0:
            raise Empty("no item")
        return self.data[self.pointer]
    
    def is_empty(self):
        return self.count==0

    def __len__(self):
        return self.count
    
    def __str__(self):
        return str(self.data)
    
q=Quene()
q.enquenue(1)
print(q)
q.enquenue(2)
print(q)
q.dequenue()
print(q)

[1]
[1, 2]
[None, 2]


**数组循环**
队列有一个问题,随着出列,队列的空间利用率越来越低,所以必须进行数组循环,当数组最后一个元素出列后,将指针置零,这样数组实际上变为了环形,空间得到完全利用.
In developing a more robust queue implementation, we allow the front of the queue to drift rightward, and we allow the contents of the queue to “wrap around” the end of an underlying array. We assume that our underlying array has fixed length N that is greater that the actual number of elements in the queue. New elements are enqueued toward the “end” of the current queue, progressing from the front to index N − 1 and continuing at index 0, then 1. Figure 6.6 illustrates such a queue with first element E and last element M.
E变成了头,M变成了尾
![Screenshot from 2017-03-23 21-10-30.png](https://ooo.0o0.ooo/2017/03/23/58d3c96c50506.png)
![Screenshot from 2017-03-23 21-12-56.png](https://ooo.0o0.ooo/2017/03/23/58d3c9f020b00.png)

#### 双端队列
在首尾都可以插入,删除的队列
抽象数据结构:
* D.add_first: 首增加
* D.add_last:  尾增加
* D.delete_first:首去除
* D.delete_last: 尾去除

* D.first:返回首元素
* D.last: 返回尾元素
* D.is_empty:判断是否为空
* len(D):返回item数量

In [52]:
#基于python的list实现的双端队列,大量使用python的语法特性,
#list的insert和pop(0)都是O(n)的复杂度,除非实在想不起来了,否则不要这么写
class Deque:
    def __init__(self):
        self.data=[]
    def isempty(self):
        return not self.data
    def add_first(self,item):
        self.data.insert(0,item)
    def add_last(self,item):
        self.data.append(item)
    def delete_first(self):
        self.data.pop(0)
    def delete_last(self):
        self.data.pop()
    def len(self):
        return len(self.data)
    def __str__(self):
        return str(self.data)
d=Deque()
d.add_first(1)
d.add_first(2)
print(d)
d.delete_first()
print(d)

[2, 1]
[1]


##### 使用循环数组
![Screenshot from 2017-03-24 00-07-53.png](https://ooo.0o0.ooo/2017/03/24/58d3f2ecb42e1.png)

In [50]:
#利用数组循环实现的双端队列
class DoubleEndedQueues:
    DEFAULT_CAPACITY=1
    
    def __init__(self):
        self._data=[None]*self.DEFAULT_CAPACITY
        self.front=1
        self.count=0
        
    def add_first(self,item):
        if self.count==len(self._data):
            self.resize(len(self._data)*2)
        self.front=(self.front-1)%len(self._data)
        self._data[self.front]=item
        self.count+=1
        
    def add_last(self,item):
        if self.count==len(self._data):
            self.resize(len(self._data)*2)
        back=(self.front+self.count)%len(self._data)
        self._data[back]=item
        self.count+=1
    
    def delete_first(self):
        if self.count==0:
            raise Expention("no item")
        item=self._data[self.front]
        self._data[self.front]=None
        self.front=(self.front+1)%len(self._data)
        self.count-=1
        return item
    
    def delete_last(self):
        if self.count==0:
            raise Empty("no item")
        back=(self.front+self.count-1)%len(self._data)
        item=self._data[back]
        self.count-=1
        return item
    
    def first(self):
        if self.count==0:
            raise Empty("no item")
        item=self._data[self.front]
        return item
    
    def last(self):
        if self.count==0:
            raise Empty("no item")
        back=(self.front+self.count-1)%len(self._data)
        item=self._data[back]
        return item  
    
    def is_empty(self):
        return self.count==0
    
    def __len__(self):
        return self.count
    
    def resize(self,size):
        data=[None]*size
        i=0
        while not self.is_empty():
            #将元素全部出列,再依次进列,目的是将队首指针设为0
            data[i]=self.delete_first()
            i+=1
        self._data=data
        self.count=i
        self.front=0
            
    def __str__(self):
        return str(self._data)

d=DoubleEndedQueues()
print(d)
d.add_first(1)
print(d)
d.add_first(2)
print(d)
d.add_first(3)
print(d)
print(d.delete_first())
print(d.delete_first())
print(d.delete_first())
#这是利用数组循环实现的双端队列,print出的顺序与实际顺序是不一致的.

[None]
[1]
[1, 2]
[2, 1, None, 3]
3
2
1
